# LSTM
2023-02-10

> Reference: https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html


### Part 1. LSTM with toy example

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

`torch.nn` - `LSTM cell`

* `input_size` : The number of expected features in the input x

* `hidden_size` : The number of features in the hidden state h
```
lstm = nn.LSTM(input_size, hidden_size)
```



In [2]:
# input_size: 3, hidden_size: 3 으로 설정하여 LSTM cell 을 생성
lstm = nn.LSTM(3, 3)  

LSTM cell 을 생성한 후에는, 입력으로 들어갈 input x, hidden state h, cell state c 를 생성해야 한다.

위에서 정한 input_size 와 hidden_size 를 고려하여 inputs 와 hidden (h 와 c) 을 생성

In [3]:
# sequence length 가 5 인 input을 생성
# 이때, input_size 를 3 으로 설정했으므로, 3 차원 벡터 5개를 생성
inputs = [torch.randn(1, 3) for _ in range(5)] 

# lstm 은 input x 와 hidden state h 를 입력으로 받기 때문에, hidden state 도 생성
# 이때, hidden_size 를 3 으로 설정했으므로, 3 차원 벡터를 생성
# lstm 의 입력으로 들어가는 h 는 RNN 에서의 hidden state 와, lstm 에서 등장한 개념인 cell state 로 구성되어 있기 때문에
# hidden 은 3 차원 벡터 2개로 구성되어야 한다.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))

**방법 1**: Sequence length 가 5 인 input 에 대하여 한 번에 하나의 element 를 lstm cell 에 통과시킨다.

In [4]:
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)

**방법 2**: 전체 시퀀스를 한번에 통과시키는 방법

LSTM 이 반환하는 출력의 첫 번째 값은 전체 시퀀스에 대한 통과한 hidden state 이고, 두 번째 값은 마지막 step 의 hidden state 이다. 

In [5]:
inputs = torch.cat(inputs).view(len(inputs), 1, -1) # 방법 2 를 적용하기 위해 input 을 list 가 아닌 하나의 tensor 로 concat 해줍니다.
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # 방법 2 를 적용하기 위해 hidden 을 다시 초기화합니다.
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward0>)
(tensor([[[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward0>), tensor([[[-0.9825,  0.4715, -0.0633]]], grad_fn=<StackBackward0>))


### Part 2. LSTM for Part-of-Speech Tagging

LSTM 을 이용해 Part-of-Speech (PoS) Tagging 을 하기 위해 학습 데이터

* `training_data` 에는 단어 시퀀스와 각 단어의 품사 태그가 필요하다.

* `word_to_ix`: 모델의 입력으로 사용하기 위해 각 단어를 id 로 mapping 한다.

* `tag_to_ix`: 품사 태그 또한 id 로 mapping 한다.

In [40]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    # Tags are: DET - determiner; NN - noun; V - verb
    # For example, the word "The" is a determiner
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
# For each words-list (sentence) and tags-list in each tuple of training_data
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:  # word has not been assigned an index yet
            word_to_ix[word] = len(word_to_ix)  # Assign each word with a unique index
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}  # Assign each tag with a unique index

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


Embedding layer, output layer, lstm cell 을 포함한 LSTMTagger 모듈을 정의

`embeds`: input id 를 embedding layer 로 encode 하여 input 에 해당하는 embedding 생성

`lstm_out`: embedding 을 lstm 에 통과하여 전체 시퀀스에 대한 hidden state 를 저장 

`tag_space`: lstm 의 output 인 hidden 을 이용해 존재하는 tag (DET, NN, V) 공간으로 linear transform 함

`tag_scores`: 이후 softmax 를 적용하여 각 tag 가 될 score 를 측정

In [7]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

정의한 LSTMTagger 클래스를 이용해 model 을 build 하고, 학습에 필요한 loss 함수와 optimizer 를 선언

In [8]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

모델을 학습하기 전 tag score 를 확인

In [9]:
# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)


tensor([[-1.1389, -1.2024, -0.9693],
        [-1.1065, -1.2200, -0.9834],
        [-1.1286, -1.2093, -0.9726],
        [-1.1190, -1.1960, -0.9916],
        [-1.0137, -1.2642, -1.0366]])


training data 를 이용해 모델을 학습

input 을 LSTMTagger 에 통과시켜 각 단어의 PoS tag 를 예측하고, 정답 tag 와 비교하여 loss 를 계산한 후 loss 를 backpropagate 하여 모델 파라미터를 업데이트함.

In [10]:

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

학습 데이터를 이용해 모델이 잘 학습되었는지 확인

In [11]:
# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[-0.0462, -4.0106, -3.6096],
        [-4.8205, -0.0286, -3.9045],
        [-3.7876, -4.1355, -0.0394],
        [-0.0185, -4.7874, -4.6013],
        [-5.7881, -0.0186, -4.1778]])


학습 데이터만을 이용해 모델을 학습하고, 평가하면서 발생할 수 있는 문제점은 일반화 성능 평가가 어렵다는 것이다. 

새로운 데이터set에 대한 학습의 성능을 보장하지 못하므로 Test set이 필요하다.

## **GRU**

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [3]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    # Tags are: DET - determiner; NN - noun; V - verb
    # For example, the word "The" is a determiner
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
# For each words-list (sentence) and tags-list in each tuple of training_data
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:  # word has not been assigned an index yet
            word_to_ix[word] = len(word_to_ix)  # Assign each word with a unique index
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}  # Assign each tag with a unique index

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [4]:
class GRUTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(GRUTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        gru_out, _ = self.gru(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(gru_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [5]:
model = GRUTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [6]:
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

tensor([[-1.1567, -0.9029, -1.2726],
        [-1.2209, -0.7717, -1.4154],
        [-1.1029, -0.9798, -1.2287],
        [-1.0932, -1.1332, -1.0704],
        [-1.2487, -0.8816, -1.2073]])


In [7]:
for epoch in range(300):
    for sentence, tags in training_data:
        model.zero_grad()

        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        tag_scores = model(sentence_in)

        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

In [8]:
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

tensor([[-0.0442, -3.2533, -5.3893],
        [-4.6415, -0.0098, -8.9834],
        [-3.9786, -8.1163, -0.0192],
        [-0.0283, -4.4065, -4.1543],
        [-4.3755, -0.0135, -7.0922]])
